In [0]:
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField,StructType, StringType, IntegerType, FloatType, LongType, DateType

# from pyspark.context import SparkContext
# sc = SparkContext.getOrCreate()

# sc = spark.sparkContext
# print(sc)

spark = SparkSession.builder.appName("scd2").getOrCreate()
print(spark)

In [0]:
from datetime import datetime, date
from pyspark.sql.types import DataType,TimestampType
data = [
    (101, 'abhishek', 'Samsung', 40000, date(2021, 1, 1), date(2022, 1, 5)),
    (102, 'dk', 'Indev Consultancy', 45000, date(2021, 1, 14), date(9999, 1, 1)),
    (103, 'rajan', 'Capgemini', 28500, date(2021, 8, 1), date(9999, 1, 1)),
    (104, 'vikash', 'Product', 55000, date(2021, 9, 1), date(9999, 1, 1))
]

schema = StructType([
    StructField('id', IntegerType(), True),
    StructField('name', StringType(), True),
    StructField('company', StringType(), True),
    StructField('sal', IntegerType(), True),
    StructField('start_date', DateType(), True),
    StructField('end_date', DateType(), True)
])

df = spark.createDataFrame(data, schema)
df.display()

In [0]:
data = [
    (101, 'abhishek', 'Samsung', 40000, date(2021, 1, 1)),
    (102, 'dk', 'Accenture', 56000, date(2021, 6, 14)),
    (103, 'rajan', 'Capgemini', 55000, date(2023, 12, 7)),
    (105, 'saurabh', 'Kpi tech', 35000, date(2024, 6, 11))
]

schema = StructType([
    StructField('id', IntegerType(), True),
    StructField('name', StringType(), True),
    StructField('company', StringType(), True),
    StructField('sal', IntegerType(), True),
    StructField('start_date', DateType(), True)
])

df1 = spark.createDataFrame(data, schema)
df1.display()

In [0]:
df = df.withColumn('map_has',xxhash64(col('id'), col('name'), col('company'), col('sal')))
df1 = df1.withColumn('map_has',xxhash64(col('id'), col('name'), col('company'), col('sal')))
display(df)
display(df1)

In [0]:
df_new = df1.join(df,['map_has'],'leftanti')
df_new = df_new.drop('map_has')
df = df.drop('map_has')
display(df_new)

In [0]:
df_new = df_new.withColumn('end_date',lit(date(9999,1,1)))
df = df.union(df_new)
display(df)

In [0]:
from pyspark.sql.window import Window 
win = Window.partitionBy('id').orderBy(('start_date'))
df = df.withColumn('rank', row_number().over(win))
display(df)

In [0]:
df.show()

In [0]:
%python
from pyspark.sql.functions import col, when, lag

win = Window.partitionBy('id').orderBy('start_date')
df = df.withColumn('end_date', when(col('rank')==2,lag(col('start_date'), 1).over(win)) \
    .otherwise(col('end_date')))

df = df.drop('rank','lag')
display(df)